In [ ]:
!pip install pandas

In [ ]:
!pip install networkx

In [ ]:
!pip install joblib

In [ ]:
!pip install gensim

In [1]:
from network import CNetwork

In [2]:
import pandas as pd
import numpy as np

In [3]:
from utils import verifyDir

In [4]:
name_dataset = "dataset_1"
dataset_path = "datasetsv2/"
length_cut = 1000
random_flag = True

In [5]:
auxiliar_path = 'auxiliar_folder/' + name_dataset   + '/'
verifyDir(auxiliar_path)

In [6]:
df = pd.read_csv(dataset_path + name_dataset + ".csv")

In [7]:
df.head(5)

,label,text,book
0,Pelham Grenville,"\n\n\n\n\n\n\n\nProduced by Christine Gehring,...",Right Ho Jeeves
1,Pelham Grenville,"\n\n\n\n\n\n\n\nProduced by Suzanne L. Shell, ...",Tales Of St Austin
2,Joseph Conrad,And that last\n\nword was the single word of ...,Victory
3,Joseph Conrad,\n\n\n\nIf I have ever had these gifts in any ...,Under Western Eyes
4,Bram Stoker,\n\n\n\n Lond...,The Lady Of The Shroud


In [8]:
from utils.text_processing import get_min_len_corpus

In [9]:
print("Min Length:", get_min_len_corpus(list(df["text"])))

Min Length: 47826


In [10]:
from utils.text_processing import get_corpus, get_random_corpus

In [11]:
texts = list(df['text'])

In [12]:
corpus, segmented_corpus = get_corpus(texts, length_cut)

In [13]:
from utils.text_processing import remove_puntuaction

In [14]:
segmented_corpus = [[[remove_puntuaction(w) for w in t] for t in p] for p in segmented_corpus ]

In [34]:
selected_corpus, words_features, word_index, index_word = get_random_corpus(segmented_corpus, tokenized=True)

['He', 'also', 'called', 'me', 'an', 'opprobrious', 'name', 'I', 'raised', 'the', 'eyebrows', 'again', 'Come', 'come', 'Tuppy', 'dont', 'let', 'us', 'let', 'this', 'little', 'chat', 'become', 'acrid', 'Is', 'acrid', 'the', 'word', 'I', 'want', 'I', 'couldnt', 'say', 'he', 'replied', 'beginning', 'to', 'sidle', 'round', 'the', 'bench', 'I', 'saw', 'that', 'anything', 'I', 'might', 'wish', 'to', 'say', 'must', 'be', 'said', 'quickly', 'Already', 'he', 'had', 'sidled', 'some', 'six', 'feet', 'And', 'though', 'by', 'dint', 'of', 'sidling', 'too', 'I', 'had', 'managed', 'to', 'keep', 'the', 'bench', 'between', 'us', 'who', 'could', 'predict', 'how', 'long', 'this', 'happy', 'state', 'of', 'affairs', 'would', 'last', 'I', 'came', 'to', 'the', 'point', 'therefore', 'I', 'think', 'I', 'know', 'whats', 'on', 'your', 'mind', 'Tuppy', 'I', 'said', 'If', 'you', 'were', 'in', 'those', 'bushes', 'during', 'my', 'conversation', 'with', 'the', 'recent', 'Angela', 'I', 'dare', 'say', 'you', 'heard', 'w

In [35]:
len(selected_corpus)

78

In [36]:
labels = list(df['label'])

In [37]:
total_classes = list(set(labels))  ## or author
print("Total classes: {}".format(len(total_classes)))
number_books = (df[df['label'] == total_classes[0]]).shape[0]
print("Total entities for each class in train: {}".format(number_books))
dict_categories = {cat: index for index, cat in enumerate(total_classes)}

Total classes: 13
Total entities for each class in train: 6


In [38]:
y = [dict_categories[y] for y in labels]

In [39]:
total_classes

['Charles Dickens',
 'Mark Twain',
 'Charles Darwin',
 'Arthur Conan Doyle',
 'Bram Stoker',
 'Jane Austen',
 'Allan Poe',
 'Thomas Hardy',
 'Joseph Conrad',
 'George Eliot',
 'Pelham Grenville',
 'Daniel Defoe',
 'Hector Hugh']

In [40]:
def read_embeddings(fname):
    with open(fname) as f:
        f.readline()
        names = []
        data = []
        for line in f:
            line = line.strip().split(",")
            names.append(line[0])
            data.append([float(v) for v in line[1:]])
        return names, data

In [41]:
def organize_data(graph2label, embed_vnames, embed_vecs):
    N = len(graph2label)
    data = [0 for _ in range(N)]
    labels = [0 for _ in range(N)]
    for i in range(len(graph2label)):
        idx = embed_vnames[i]
        data[int(idx)] = embed_vecs[i]
        labels[int(idx)] = graph2label[idx]
    return np.array(data), np.array(labels)

In [42]:
from utils.igraph2json import igraph2json1
import os

In [43]:
def graph2vec1(path, operating_system, networks, labels):
    networks_dir = path + 'dataset/'
    emb_file = path + 'features/nci1.csv'
    graph2label = {}
    try: 
        os.mkdir(in_network)
        os.mkdir(path + 'features/')
    except:
        print("Existe")
    for (i, netw), label in zip(enumerate(networks), labels):
        graph2label[str(i)] = label
        igraph2json1(netw, networks_dir + str(i) + ".json")
    if operating_system == 'linux':
        path_command = 'python ./graph2vec/src/graph2vec.py --input-path ' + networks_dir + ' --output-path ' + emb_file + ' --dimensions ' + str(512)
    os.system(path_command)
    embed_vnames, embed_vecs = read_embeddings(emb_file)
    X, y = organize_data(graph2label, embed_vnames, embed_vecs)
    return X, y


In [44]:
def graph2vec(path, operating_system, networks):
    in_network = path + 'dataset/'
    extra_file = path + 'features/nci1.csv'
    try: 
        os.mkdir(in_network)
        os.mkdir(path + 'features/')
    except:
        print("Existe")
    for i, netw in enumerate(networks):
        igraph2json1(netw, in_network + str(i) + ".json")
    if operating_system == 'linux':
        path_command = 'python graph2vec/src/graph2vec.py --input-path ' + in_network + ' --output-path ' + extra_file + ' --dimensions ' + str(512)
    os.system(path_command)
    g2v = pd.read_csv(extra_file, sep=',', lineterminator='\n')
    g2v.set_index("type")
    g2v.drop("type", axis="columns", inplace=True)
    all_network_features = [g2v.loc[i].values for i, _ in enumerate(networks)]
    print("graph2vec", len(all_network_features))
    return all_network_features

In [45]:
def get_graph2vec_features(sequences):
    network_features = pd.DataFrame()
    nets = []
    for text in sequences:
        obj = CNetwork(text, model=None, index_word=index_word, percentages=None, path=auxiliar_path)
        network = obj.create_network()
        nets.append(network)
    X = np.array(graph2vec("", "linux", nets))
    return X

In [46]:
X = get_graph2vec_features(selected_corpus)

Nodes: 470 - Edges: 892
Nodes: 475 - Edges: 905
Nodes: 494 - Edges: 919
Nodes: 508 - Edges: 907
Nodes: 426 - Edges: 869
Nodes: 380 - Edges: 880
Nodes: 517 - Edges: 916
Nodes: 467 - Edges: 911
Nodes: 414 - Edges: 872
Nodes: 466 - Edges: 906
Nodes: 457 - Edges: 893
Nodes: 418 - Edges: 855
Nodes: 402 - Edges: 827
Nodes: 411 - Edges: 867
Nodes: 486 - Edges: 925
Nodes: 392 - Edges: 842
Nodes: 490 - Edges: 914
Nodes: 491 - Edges: 907
Nodes: 450 - Edges: 881
Nodes: 452 - Edges: 898
Nodes: 497 - Edges: 891
Nodes: 444 - Edges: 903
Nodes: 428 - Edges: 903
Nodes: 482 - Edges: 919
Nodes: 445 - Edges: 868
Nodes: 516 - Edges: 928
Nodes: 451 - Edges: 888
Nodes: 499 - Edges: 901
Nodes: 451 - Edges: 902
Nodes: 467 - Edges: 903
Nodes: 373 - Edges: 788
Nodes: 480 - Edges: 907
Nodes: 483 - Edges: 863
Nodes: 426 - Edges: 909
Nodes: 369 - Edges: 817
Nodes: 424 - Edges: 914
Nodes: 432 - Edges: 883
Nodes: 494 - Edges: 927
Nodes: 506 - Edges: 897
Nodes: 488 - Edges: 925
Nodes: 510 - Edges: 918
Nodes: 481 - Edg

  0%|          | 0/78 [00:00<?, ?it/s]


Feature extraction started.



100%|██████████| 78/78 [00:00<00:00, 134.79it/s]



Optimization started.

graph2vec 78


In [47]:
print("Lenght of features:", X.shape)

Lenght of features: (78, 512)


# Training with SVM

In [48]:
from classifierv2 import getClassifier, getClassMetrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split

In [53]:
RANDOM_SEED = 20
FRAC_TRAIN = 0.8

In [54]:
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=(1.0 - FRAC_TRAIN), random_state=RANDOM_SEED)

In [55]:
from sklearn.svm import SVC

In [56]:
svc = SVC(kernel='linear', probability=True)
svc.fit(X_train, y_train)

predicted = svc.predict(X_val)
print(y_val)
print(predicted)
#print accuracy_score
print("Accuracy : " + str(accuracy_score(y_val, predicted)))

print("Micro f-measure " + str(f1_score(y_val, predicted, average='micro')))

[7, 9, 10, 0, 12, 11, 5, 11, 2, 1, 6, 8, 10, 3, 4, 3]
[9 9 4 5 8 4 4 9 9 9 1 4 8 4 5 8]
Accuracy : 0.0625
Micro f-measure 0.0625
